# Stock Price Direction Prediction - Stationary Dataset Generation

This notebook generates two datasets for predicting stock price direction:
1. **Basic Dataset**: OHLCV data with log return and interday volitility measures with binary target (candlestick)


2. **Enhanced Dataset**: OHLCV data + technical indicators with binary target

## Setup and Data Collection
- Tickers: Top 50 S&P 500 stocks by market cap
- Date Range: 2010-01-01 to 2024-12-31
- Train/Test Split: 2010-2021 (Train), 2022-2024 (Test)

In [47]:
# Import required libraries
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
import time
warnings.filterwarnings('ignore')

print("Libraries imported successfully")

Libraries imported successfully


Pandas methods used in this notebook — quick reference

Below are concise, actionable descriptions of the DataFrame/Series methods and operations that appear in this notebook. Each item shows what it does and a brief example of the effect.

- shift(n)
  - Moves values up/down along the index. Commonly used to compare current row with next/previous row.
  - Example: df['Close'].shift(-1) gives next-day close.

- astype(type)
  - Casts dtype of a Series (or DataFrame) to the given type.
  - Example: (next_close > df['Close']).astype(int) converts boolean to 0/1.

- df.columns (assignment)
  - Replace column labels. Used here to flatten MultiIndex columns by joining parts or taking the last part.
  - Example: df.columns = [f"{p}_{t}" for p,t in df.columns]

- isinstance(df.columns, pd.MultiIndex)
  - Checks whether columns have multiple levels (e.g., (Price, Ticker)). If true, you may want to flatten or stack.

- get_level_values(level)
  - Returns the labels for a specific MultiIndex level (useful to find empty tickers or particular level values).
  - Example: df.columns.get_level_values('Ticker')

- stack(level)
  - Moves one column-level into the row index, turning wide MultiIndex columns into long rows. Use when you want a tidy table with a Ticker column.
  - Example: df.stack(level='Ticker') → index becomes (original_index, Ticker) and price labels stay as columns.

- reset_index()
  - Converts index levels into columns and resets index to default integer index.
  - Often used after stack() to get a flat DataFrame: df.stack(...).reset_index()

- sort_values(by)
  - Sorts rows by one or more columns. Important before groupby operations that assume ordering.
  - Example: df.sort_values(['Ticker','Date'])

- reset_index(drop=True)
  - Reset index and drop the old index instead of adding it as a column.

- groupby(...).apply(func)
  - Group rows and apply a function to each group. Useful for per-ticker operations, e.g., remove last row for each ticker: df.groupby('Ticker').apply(lambda x: x.iloc[:-1])

- iloc / loc
  - iloc: integer-location based selection. loc: label-based selection. Both used to select rows/columns precisely.
  - Example: x.iloc[:-1] takes all but the last row of group x. dataset2.loc[mask, 'SMA_10'] assigns values to rows matching mask.

- unique()
  - Returns unique values in a Series (e.g., list of tickers).

- isnull().sum()
  - Counts missing values per column.

- dropna()
  - Drops rows (or columns) containing NA. Here used to remove rows missing indicator values after rolling/ewm computations.

- rolling(window).mean()
  - Computes rolling statistics (SMA) over a window of rows.
  - Example: df['Close'].rolling(10).mean() gives 10-day SMA.

- diff()
  - First discrete difference on Series: current - previous. Used to compute gains/losses for RSI.

- where(condition, other)
  - Keeps values where condition True, else replaces with other. Useful to separate gains (positive diffs) from losses.

- ewm(span, adjust=False).mean()
  - Exponential weighted mean (EMA). Used for MACD and signal smoothing.

- drop_duplicates()
  - Remove duplicate rows (not used heavily here but commonly used when combining datasets).

- to_csv / to_parquet
  - Persist DataFrame to disk in CSV or Parquet formats.

- melt(id_vars, var_name, value_name)
  - Unpivot a DataFrame from wide to long format. Use when you flattened column names like 'Close_AAPL' and want rows per ticker.
  - Example: df.reset_index().melt(id_vars='Date')

- str.rsplit(sep, n=1, expand=True)
  - Split string column from right into parts. Useful to split 'Close_AAPL' into ['Close','AAPL'].

- copy()
  - Create an explicit copy of a DataFrame to avoid modifying the original view in-place.

- head(n) / shape / columns / value_counts() / mean()
  - Utility methods: head shows top rows, shape gives (rows, cols), columns lists column names, value_counts gives counts per value, mean gives average (useful for target balance).

Tip: for tidy machine learning datasets prefer the long form (one row per Date+Ticker) produced by stacking or melting — columns: Date, Ticker, Open, High, Low, Close, Volume, indicators..., Target.

## Step 1: Get Top 50 S&P 500 Stocks by Market Cap

In [48]:
# Get S&P 500 tickers from Wikipedia (robust)
top_50_tickers = ['AAPL', 'MSFT']
                  #, 'AMZN', 'GOOGL', 'BRK-B', 'JPM', 'JNJ', 'PG', 'V', 'UNH']
# yfinance expects '-' instead of '.' for tickers like BRK.B
top_50_tickers = [t.replace('.', '-').strip() for t in top_50_tickers]

print(f"Using {len(top_50_tickers)} tickers for the dataset:")
print(top_50_tickers)

Using 2 tickers for the dataset:
['AAPL', 'MSFT']


## Step 2: Download Historical Data

In [49]:
# Define date ranges
start_date = '2010-01-01'
end_date = '2024-12-31'

print(f"Downloading historical data from {start_date} to {end_date}...\n")

# Download data for all tickers
all_data = []


if not top_50_tickers:
    raise RuntimeError("No tickers available to download. Check the Wikipedia scraping step.")

for ticker in top_50_tickers:
    try:
        print(f"Downloading {ticker}...", end=' ')
        
        df = yf.download(ticker, start=start_date, end=end_date, progress=False)
        
        
        if not df.empty:
            # 1. FIX: Flatten the columns FIRST (before appending)
            if isinstance(df.columns, pd.MultiIndex):
                df.columns = df.columns.droplevel('Ticker')
            
            # 2. Add Ticker column (Crucial so you can distinguish AAPL vs MSFT later)
            df['Ticker'] = ticker
            
            # 3. NOW calculate your features (using clean column names like 'Close')
            next_close = df['Close'].shift(-1)
            df['Target'] = (next_close > df['Close']).astype(int)
            next_close = df['Close'].shift(-1)
            df['Target'] = (next_close > df['Close']).astype(int)
            # 1. Close: The main trend (Log Return from Yesterday Close to Today Close)
            df['Log_Ret_Close'] = np.log(df['Close'] / df['Close'].shift(1))
            
            # 2. Open: The Overnight Move (Log Return from Yesterday Close to Today Open)
            df['Log_Ret_Overnight'] = np.log(df['Open'] / df['Close'].shift(1))
            
            # 3. High/Low: The Intraday Volatility (Normalized to Open)
            # "How much did it swing relative to where it started?"
            df['High_Rel'] = df['High'] / df['Open']
            df['Low_Rel'] = df['Low'] / df['Open']
            
            # 4. Volume: Log Change (Stationary growth)
            df['Log_Vol_Change'] = np.log(df['Volume'] / df['Volume'].shift(1))
            
            all_data.append(df.dropna())
            print(f"OK ({len(df)} rows)")
            
            if isinstance(df.columns, pd.MultiIndex):
                df.columns = df.columns.droplevel('Ticker')
                print(df.columns)
            
        else:
            print("No data available")
    except Exception as e:
        print("Error:", e)
    time.sleep(0.5)  # be polite to the data provider


In [50]:

# Combine all data
if not all_data:
    raise RuntimeError("No data was downloaded for any tickers. Aborting.")


raw_data = all_data
print('raw_data ---------------------------------------------')
print(raw_data[1])


raw_data ---------------------------------------------
Price            Close        High         Low        Open    Volume Ticker  \
Date                                                                          
2010-01-05   23.180838   23.285662   22.941243   23.098478  49749600   MSFT   
2010-01-06   23.038572   23.270680   22.851389   23.120932  58182400   MSFT   
2010-01-07   22.798985   22.986168   22.604313   22.933756  50559700   MSFT   
2010-01-08   22.956211   23.120932   22.641742   22.671692  51197400   MSFT   
2010-01-11   22.664215   23.031095   22.551905   22.993658  68754700   MSFT   
...                ...         ...         ...         ...       ...    ...   
2024-12-23  432.871429  435.258308  430.464641  434.353277  19152500   MSFT   
2024-12-24  436.929108  437.197652  431.817213  432.274691   7164500   MSFT   
2024-12-26  435.715790  438.530341  434.243897  436.680490   8194200   MSFT   
2024-12-27  428.177216  432.841588  424.020066  432.224981  18117700   MSFT 

## Step 4: Create Dataset 1 - Basic OHLCV Data

In [51]:
# Dataset 1: Basic OHLCV + Target
features_to_keep = [
    'Log_Ret_Close',
    'Log_Ret_Overnight',
    'High_Rel',
    'Low_Rel',
    'Log_Vol_Change',
    'Target'  # Don't forget the target!
]

dataset1 = [data[features_to_keep].copy(deep=True) for data in raw_data]
print(f"Dataset 1 (Basic) created")
print(dataset1[0].head(10))

Dataset 1 (Basic) created
Price       Log_Ret_Close  Log_Ret_Overnight  High_Rel   Low_Rel  \
Date                                                               
2010-01-05       0.001727       2.752930e-03  1.004613  0.993709   
2010-01-06      -0.016034       1.058689e-07  1.003965  0.983067   
2010-01-07      -0.001850       3.690556e-03  1.001181  0.987249   
2010-01-08       0.006626      -1.330695e-03  1.008084  0.994104   
2010-01-11      -0.008860       3.860972e-03  1.000940  0.979558   
2010-01-12      -0.011440      -4.388310e-03  1.002773  0.986759   
2010-01-13       0.014007       7.221870e-04  1.014721  0.981864   
2010-01-14      -0.005809      -2.566949e-03  1.001666  0.994812   
2010-01-15      -0.016853       7.136682e-03  1.003176  0.976011   
2010-01-19       0.043288       1.158698e-02  1.032929  0.994768   

Price       Log_Vol_Change  Target  
Date                                
2010-01-05        0.198111       0  
2010-01-06       -0.086261       0  
2010-01-0

## Step 5: Calculate Technical Indicators for Dataset 2

Technical indicators to calculate:
- **Simple Moving Averages (SMA)**: 10-day, 50-day, 200-day
- **Relative Strength Index (RSI)**: 14-day period
- **MACD**: 12, 26, 9 parameters

In [52]:
def calculate_sma(data, window):
    """Calculate Simple Moving Average"""
    return data.rolling(window=window).mean()

def calculate_rsi(data, window=14):
    """Calculate Relative Strength Index"""
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def calculate_macd(data, fast=12, slow=26, signal=9):
    """Calculate MACD and Signal line"""
    ema_fast = data.ewm(span=fast, adjust=False).mean()
    ema_slow = data.ewm(span=slow, adjust=False).mean()
    macd = ema_fast - ema_slow
    signal_line = macd.ewm(span=signal, adjust=False).mean()
    return macd, signal_line

print("Technical indicator functions defined")

Technical indicator functions defined


In [53]:
# Create Dataset 2 with technical indicators
dataset2 = [data.copy(deep=True) for data in raw_data]

print("Calculating technical indicators for each ticker...\n")
features_to_keep = [
    'Log_Ret_Close',
    'Log_Ret_Overnight',
    'High_Rel',
    'Low_Rel',
    'Log_Vol_Change',
    'SMA-10',
    'SMA-50',
    'SMA-200',
    'RSI',
    'MACD',
    'Target'  # Don't forget the target!
    
]
for i in range(len(dataset2)):
    ticker_data = dataset2[i]
    ticker_data['SMA-10'] = calculate_sma(ticker_data['Close'], 10)
    ticker_data['SMA-50'] = calculate_sma(ticker_data['Close'], 50)
    ticker_data['SMA-200'] = calculate_sma(ticker_data['Close'], 200)
    
    ticker_data['RSI'] = calculate_rsi(ticker_data['Close'])
    
    ticker_data['MACD'], ticker_data['Signal_Line'] = calculate_macd(ticker_data['Close'])
    
    dataset2[i] = ticker_data[features_to_keep].copy().dropna()
    
    print(dataset2[i].head(5))
    

    

Calculating technical indicators for each ticker...

Price       Log_Ret_Close  Log_Ret_Overnight  High_Rel   Low_Rel  \
Date                                                               
2010-10-19      -0.027126          -0.046999  1.034179  0.988860   
2010-10-20       0.003355          -0.001585  1.016990  0.993107   
2010-10-21      -0.003258           0.005876  1.007619  0.982200   
2010-10-22      -0.006645          -0.001454  1.003138  0.991038   
2010-10-25       0.004446           0.005255  1.008120  0.997897   

Price       Log_Vol_Change    SMA-10    SMA-50   SMA-200        RSI      MACD  \
Date                                                                            
2010-10-19        0.120339  9.030483  8.140073  7.376566  69.540600  0.321368   
2010-10-20       -0.535526  9.094484  8.170736  7.390984  74.786955  0.320406   
2010-10-21       -0.268935  9.155365  8.206323  7.405762  75.092795  0.313584   
2010-10-22       -0.391586  9.195554  8.239721  7.420291  77.7373

## Step 6: Clean Data - Drop Rows with Missing Values

## Step 7: Split Data into Train and Test Sets

- **Training**: 2010-2021
- **Testing**: 2022-2024

In [54]:
# Define split date
split_date = '2022-01-01'

# Combine the DFs

dataset1_full = pd.concat(dataset1)


dataset2_full = pd.concat(dataset2)

print(dataset1_full.head(5))
print(dataset2_full.head(5))

# Split Dataset 1

dataset1_train = dataset1_full[dataset1_full.index < split_date].copy()
dataset1_test = dataset1_full[dataset1_full.index >= split_date].copy()

# Split Dataset 2
dataset2_train = dataset2_full[dataset2_full.index < split_date].copy()
dataset2_test = dataset2_full[dataset2_full.index >= split_date].copy()

print(f"{'='*60}")
print(f"Dataset 1 (Basic OHLCV)")
print(f"{'='*60}")
print(f"Training set: {dataset1_train.shape[0]:,} rows ({dataset1_train.index.min()} to {dataset1_train.index.max()})")
print(f"Testing set:  {dataset1_test.shape[0]:,} rows ({dataset1_test.index.min()} to {dataset1_test.index.max()})")
print(f"\nTarget distribution - Train: {dataset1_train['Target'].mean()*100:.2f}% positive")
print(f"Target distribution - Test:  {dataset1_test['Target'].mean()*100:.2f}% positive")

print(f"\n{'='*60}")
print(f"Dataset 2 (With Technical Indicators)")
print(f"{'='*60}")
print(f"Training set: {dataset2_train.shape[0]:,} rows ({dataset2_train.index.min()} to {dataset2_train.index.max()})")
print(f"Testing set:  {dataset2_test.shape[0]:,} rows ({dataset2_test.index.min()} to {dataset2_test.index.max()})")
print(f"\nTarget distribution - Train: {dataset2_train['Target'].mean()*100:.2f}% positive")
print(f"Target distribution - Test:  {dataset2_test['Target'].mean()*100:.2f}% positive")

Price       Log_Ret_Close  Log_Ret_Overnight  High_Rel   Low_Rel  \
Date                                                               
2010-01-05       0.001727       2.752930e-03  1.004613  0.993709   
2010-01-06      -0.016034       1.058689e-07  1.003965  0.983067   
2010-01-07      -0.001850       3.690556e-03  1.001181  0.987249   
2010-01-08       0.006626      -1.330695e-03  1.008084  0.994104   
2010-01-11      -0.008860       3.860972e-03  1.000940  0.979558   

Price       Log_Vol_Change  Target  
Date                                
2010-01-05        0.198111       0  
2010-01-06       -0.086261       0  
2010-01-07       -0.146046       1  
2010-01-08       -0.063867       0  
2010-01-11        0.032138       0  
Price       Log_Ret_Close  Log_Ret_Overnight  High_Rel   Low_Rel  \
Date                                                               
2010-10-19      -0.027126          -0.046999  1.034179  0.988860   
2010-10-20       0.003355          -0.001585  1.016990  0.99

## Step 8: Save Datasets to CSV and Parquet

In [55]:
# Create data directory if it doesn't exist
import os
os.makedirs('../data', exist_ok=True)

# Save Dataset 1 (Basic)
print("Saving Dataset 1 (Basic OHLCV)...")
dataset1_train.to_csv('../data/stationary/dataset1_train.csv', index=False)
dataset1_test.to_csv('../data/stationary/dataset1_test.csv', index=False)
dataset1_train.to_parquet('../data/stationary/dataset1_train.parquet', index=False)
dataset1_test.to_parquet('../data/stationary/dataset1_test.parquet', index=False)
print("âœ“ Dataset 1 saved")

# Save Dataset 2 (Enhanced)
print("\nSaving Dataset 2 (With Technical Indicators)...")
dataset2_train.to_csv('../data/stationary/dataset2_train.csv', index=False)
dataset2_test.to_csv('../data/stationary/dataset2_test.csv', index=False)
dataset2_train.to_parquet('../data/stationary/dataset2_train.parquet', index=False)
dataset2_test.to_parquet('../data/stationary/dataset2_test.parquet', index=False)
print("âœ“ Dataset 2 saved")

print(f"\n{'='*60}")
print("All datasets saved successfully!")
print(f"{'='*60}")
print("\nFiles created in ../data/stationary/ directory:")
print("  - dataset1_train.csv / .parquet")
print("  - dataset1_test.csv / .parquet")
print("  - dataset2_train.csv / .parquet")
print("  - dataset2_test.csv / .parquet")

Saving Dataset 1 (Basic OHLCV)...
âœ“ Dataset 1 saved

Saving Dataset 2 (With Technical Indicators)...
âœ“ Dataset 2 saved

All datasets saved successfully!

Files created in ../data/stationary/ directory:
  - dataset1_train.csv / .parquet
  - dataset1_test.csv / .parquet
  - dataset2_train.csv / .parquet
  - dataset2_test.csv / .parquet


## Step 9: Summary Statistics

In [56]:
print("="*80)
print("DATASET GENERATION SUMMARY")
print("="*80)

print("\n1. DATA COLLECTION")
print(f"   - Number of stocks: {len(top_50_tickers)}")
print(f"   - Date range: 2010-01-01 to 2024-12-31")
print(f"   - Tickers: {', '.join(top_50_tickers[:10])}...")

print("\n2. DATASET 1 (Basic OHLCV)")
print(f"   Columns: {list(dataset1_train.columns)}")
print(f"   - Training: {dataset1_train.shape[0]:,} rows, {dataset1_train.shape[1]} features")
print(f"   - Testing:  {dataset1_test.shape[0]:,} rows, {dataset1_test.shape[1]} features")

print("\n3. DATASET 2 (With Technical Indicators)")
print(f"   Columns: {list(dataset2_train.columns)}")
print(f"   - Training: {dataset2_train.shape[0]:,} rows, {dataset2_train.shape[1]} features")
print(f"   - Testing:  {dataset2_test.shape[0]:,} rows, {dataset2_test.shape[1]} features")

print("\n4. TECHNICAL INDICATORS INCLUDED")
print("   - SMA_10, SMA_50, SMA_200 (Simple Moving Averages)")
print("   - RSI_14 (Relative Strength Index)")
print("   - MACD, MACD_Signal, MACD_Histogram")

print("\n5. FILE OUTPUTS")
print("   CSV and Parquet formats saved to ../data/ directory")

print("\n" + "="*80)
print("Dataset generation complete!")
print("="*80)

DATASET GENERATION SUMMARY

1. DATA COLLECTION
   - Number of stocks: 2
   - Date range: 2010-01-01 to 2024-12-31
   - Tickers: AAPL, MSFT...

2. DATASET 1 (Basic OHLCV)
   Columns: ['Log_Ret_Close', 'Log_Ret_Overnight', 'High_Rel', 'Low_Rel', 'Log_Vol_Change', 'Target']
   - Training: 6,040 rows, 6 features
   - Testing:  1,504 rows, 6 features

3. DATASET 2 (With Technical Indicators)
   Columns: ['Log_Ret_Close', 'Log_Ret_Overnight', 'High_Rel', 'Low_Rel', 'Log_Vol_Change', 'SMA-10', 'SMA-50', 'SMA-200', 'RSI', 'MACD', 'Target']
   - Training: 5,642 rows, 11 features
   - Testing:  1,504 rows, 11 features

4. TECHNICAL INDICATORS INCLUDED
   - SMA_10, SMA_50, SMA_200 (Simple Moving Averages)
   - RSI_14 (Relative Strength Index)
   - MACD, MACD_Signal, MACD_Histogram

5. FILE OUTPUTS
   CSV and Parquet formats saved to ../data/ directory

Dataset generation complete!
